# Using Twitcher Client

In [ ]:
# disable ssl warnings
import urllib3
urllib3.disable_warnings()

In [ ]:
from twitcher.client import TwitcherService
base_url = 'http://localhost:8000'
twitcher = TwitcherService(base_url, verify=False)

## Register Client APP

In [ ]:
client = twitcher.add_client_app(username='demo', password='demo', name='test1', redirect_uri='http://demo/test1')
print(client)

## Get OAuth access token to register WPS service

In [ ]:
token = twitcher.fetch_token(client_id=client['client_id'], client_secret=client['client_secret'], scope='register')
print(token)

## Register WPS service in Twitcher OWS Proxy

In [ ]:
service = twitcher.register_service(access_token=token['access_token'], name="emu_demo", url="http://localhost:5000/wps")
print(service)

## WPS GetCapabilities request

In [ ]:
import requests

url = "{}/ows/proxy/emu_demo?service=WPS&request=GetCapabilities".format(base_url)
print(url)

In [ ]:
resp = requests.get(url, verify=False)
resp.ok

## WPS DescribeProcess request

In [ ]:
url = "{}/ows/proxy/emu_demo?service=WPS&version=1.0.0&request=DescribeProcess&identifier=hello".format(base_url)
print(url)

In [ ]:
resp = requests.get(url, verify=False)
resp.ok

## WPS Execute request ... restricted access

In [ ]:
url = "{}/ows/proxy/emu_demo?service=WPS&version=1.0.0&request=Execute&identifier=hello&DataInputs=name=Tux".format(base_url)
print(url)

In [ ]:
resp = requests.get(url, verify=False)
resp.ok

Access forbidden without access token.

In [ ]:
'AccessForbidden' in resp.text

Get Access token for compute

In [ ]:
compute_token = twitcher.fetch_token(client_id=client['client_id'], client_secret=client['client_secret'], scope='compute')
print(compute_token)

Add access token to parameters

In [ ]:
url = "{}/ows/proxy/emu_demo?service=WPS&version=1.0.0&request=Execute&identifier=hello&DataInputs=name=Tux&access_token={}".format(
        base_url, compute_token['access_token'])
print(url)

In [ ]:
resp = requests.get(url, verify=False)
resp.ok

In [ ]:
'Hello Tux' in resp.text

... or add access token to headers

In [ ]:
from twitcher.client import get_headers
headers = get_headers(compute_token['access_token'])
print(headers)

In [ ]:
resp = requests.get(
    "{}/ows/proxy/emu_demo?service=WPS&version=1.0.0&request=Execute&identifier=hello&DataInputs=name=Tux".format(base_url),
    headers=headers,
    verify=False)

In [ ]:
'Hello Tux' in resp.text